In [2]:
# %matplotlib inline
# %matplotlib notebook
import tqdm, copy
import random, os
import pandas as pd
from tqdm import trange
import matplotlib.pyplot as plt
from collections import defaultdict
from overcooked_ai_py.utils import save_pickle
from human_aware_rl.utils import set_global_seed
from human_aware_rl.human.process_dataframes import *
# from human_aware_rl.static import *
import pickle as pkl

set_global_seed(1884)
import ast
import json

from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
import os
from ast import literal_eval
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas


# Helper Functions

In [4]:

def f_p1(t, a, p1_data):
    t_partial = t[int(a)-1:int(a)]
#     print('t_input', t)
    return [p1_data[int(j)]['position'][0] for j in t_partial], [p1_data[int(j)]['position'][1] for j in t_partial]
def f_p2(t, a, p2_data):
    t_partial = t[int(a)-1:int(a)]
    return [p2_data[int(j)]['position'][0] for j in t_partial], [p2_data[int(j)]['position'][1] for j in t_partial]



def arrow_p1(t, a, p1_data):
#     t_partial = t[int(a)-1:int(a)+1]
#     print('t_input', t)
    return p1_data[int(a)]['position'][0], p1_data[int(a)]['position'][1],\
            p1_data[int(a)]['orientation'][0], p1_data[int(a)]['orientation'][1]
def arrow_p2(t, a, p2_data):
    return p2_data[int(a)]['position'][0], p2_data[int(a)]['position'][1],\
            p2_data[int(a)]['orientation'][0], p2_data[int(a)]['orientation'][1]


def held_p1(t, a, p1_data):
    return p1_data[int(a)]['position'][0], p1_data[int(a)]['position'][1],\
            p1_data[int(a)]['orientation'][0], p1_data[int(a)]['orientation'][1]
def held_p2(t, a, p2_data):
    return p2_data[int(a)]['position'][0], p2_data[int(a)]['position'][1],\
            p2_data[int(a)]['orientation'][0], p2_data[int(a)]['orientation'][1]


def world_obj(t, a, objects_data):
    obj_world = objects_data[int(a)]
    if len(obj_world)==0:
        return []
    objects_list = []
    for i in range(len(obj_world)):
        obj = obj_world[i]
        name = obj_world[i]['name']
        position = obj_world[i]['position']
        if name == 'onion':
            color = 'y'
            objects_list.append((position[0], position[1], name, color))
        if name == 'dish':
            color = 'k'
            objects_list.append((position[0], position[1], name, color))
        if name == 'soup':
            if obj['is_cooking'] is True:
                color = 'r'
            elif obj['is_ready'] is True:
                color = 'g'
            else:
                color = 'orange'
            objects_list.append((position[0], position[1], name, color))
        
    return objects_list


def obj_p1(t, a, p1_data):
#     t_partial = t[int(a)-1:int(a)+1]
#     print('t_input', t)
    color = 'k'
    if p1_data[int(a)]['held_object'] is None:
        return [None, None, None, None]
    else:
        name = p1_data[int(a)]['held_object']['name']
        if name == 'dish':
            color = 'k'
        elif name == 'onion':
            color = 'y'
#         elif name == 'onion':
#             color = 'y'
    return p1_data[int(a)]['position'][0], p1_data[int(a)]['position'][1], p1_data[int(a)]['held_object']['name'], color

def obj_p2(t, a, p2_data):
    color = 'k'
    if p2_data[int(a)]['held_object'] is None:
        return [None, None, None, None]
    else:
        name = p2_data[int(a)]['held_object']['name']
        if name == 'dish':
            color = 'k'
        elif name == 'onion':
            color = 'y'
    return p2_data[int(a)]['position'][0], p2_data[int(a)]['position'][1], p2_data[int(a)]['held_object']['name'], color


# [{'name': 'soup',
#    'position': [4, 1],
#    '_ingredients': [{'name': 'onion', 'position': [4, 1]},
#     {'name': 'onion', 'position': [4, 1]}],
#    'cooking_tick': -1,
#    'is_cooking': False,
#    'is_ready': False,
#    'is_idle': True,
#    'cook_time': -1,
#    '_cooking_tick': -1}],

NORTH = (0, -1)
SOUTH = (0, 1)
EAST  = (1, 0)
WEST  = (-1, 0)
STAY = (0,0)
INTERACT = 'INTERACT'
# STAY = [0,0]

def action_p1(t, a, p1_data):
    if p1_data[int(a)] == 'INTERACT':
        act = 'I'
    else:
        x, y = p1_data[int(a)][0], p1_data[int(a)][1]
        act = 'N'
        if (x,y) == NORTH:
            act = 'N'
        if (x,y) == SOUTH:
            act = 'S'
        if (x,y) == EAST:
            act = 'E'
        if (x,y) == WEST:
            act = 'W'
        if (x,y) == STAY:
            act = 'Y'
        
    return act

def action_p2(t, a, p2_data):
    if p2_data[int(a)] == 'INTERACT':
        act = 'I'
    else:
        x, y = p2_data[int(a)][0], p2_data[int(a)][1]
        act = 'N'
        if (x,y) == NORTH:
            act = 'N'
        if (x,y) == SOUTH:
            act = 'S'
        if (x,y) == EAST:
            act = 'E'
        if (x,y) == WEST:
            act = 'W'
        if (x,y) == STAY:
            act = 'Y'
        
    return act





In [5]:
def json_eval(s):
    json_acceptable_string = s.replace("'", "\"")
    d = json.loads(json_acceptable_string)
    return d

In [9]:
def import_2019_data():
    import pickle as pkl
    hh_all_2019_file = '/Users/michellezhao/Documents/overcooked_research/human_aware_rl/human_aware_rl/static/human_data/cleaned/2019_hh_trials_all.pickle'

    with open(hh_all_2019_file,'rb') as file:
        humans_2019_file = pkl.load(file)

    humans_2019_file.to_csv('humans_all_2019.csv')
    old_trials = humans_2019_file
    return old_trials

def import_2020_data():
    import pickle as pkl
    hh_all_2020_file = '/Users/michellezhao/Documents/overcooked_research/human_aware_rl/human_aware_rl/static/human_data/cleaned/2020_hh_trials_all.pickle'

    with open(hh_all_2020_file,'rb') as file:
        humans_2020_file = pkl.load(file)

    humans_2020_file.to_csv('humans_all_2020.csv')
    new_trials = humans_2020_file
    return new_trials






In [10]:
def import_cleaned_data(filename):
    

    with open(filename,'rb') as file:
        data = pkl.load(file)

    return data




In [53]:
def get_cleaned_fc_data_train():
    filename = '../data/human/anonymized/clean_train_trials.pkl'
    clean_data = import_cleaned_data(filename)
    fc_data = clean_data[clean_data['layout_name']=='random0']
    print(fc_data.groupby(['run','workerid_num', 'reward_norm_total']).size().reset_index().rename(columns={0:'count'}))
    return fc_data

def get_cleaned_fc_data_test():
    filename = '../data/human/anonymized/clean_test_trials.pkl'
    clean_data = import_cleaned_data(filename)
    fc_data = clean_data[clean_data['layout_name']=='random0']
    print(fc_data.groupby(['run','workerid_num', 'reward_norm_total']).size().reset_index().rename(columns={0:'count'}))
    return fc_data


    
    
    
    
    
    
    
    

# Make Comparison

In [11]:
old_trials = import_2019_data()

In [12]:

name = 'random0'
title = 'Forced Coord'

# name = 'random0'
# title = 'Forced Coordination'
old_trials = import_2019_data()
layout_trials = old_trials[old_trials['layout_name']==name]['trial_id'].unique()

In [14]:
old_trials.columns

Index(['Unnamed: 0', 'cur_gameloop', 'joint_action', 'layout', 'layout_name',
       'reward', 'score', 'state', 'time_elapsed', 'time_left', 'trial_id',
       'player_0_is_human', 'player_1_is_human', 'player_0_id', 'player_1_id',
       'cur_gameloop_total', 'score_total', 'button_press',
       'button_press_total', 'timesteps_since_interact',
       'button_presses_per_timstep'],
      dtype='object')

In [13]:
layout_trials

array([  9,  14,  19,  24,  54,  59,  64,  69,  79,  89,  99, 114])

In [36]:
old_trials_team_states = old_trials[old_trials['trial_id']==9]['state'].to_numpy()

In [38]:
fc_data = get_cleaned_fc_data()
states_for_player = fc_data[fc_data['workerid_num']==4]['state'].to_numpy()

     run  workerid_num  reward_norm_total  count
0  run-1             2              480.0   1204
1  run-1             4              280.0   1204
2  run-2            15              260.0   1199
3  run-2            17              300.0   1204
4  run-2            19              400.0   1204
5  run-2            22              300.0   1136


In [46]:
eval(states_for_player[0])

{'players': [{'position': [3, 1], 'orientation': [0, -1]},
  {'position': [1, 2], 'orientation': [0, -1]}],
 'objects': {},
 'order_list': ['onion'],
 'pot_explosion': False}

In [37]:
old_trials_team_states[0]

'{"players": [{"position": [3, 1], "orientation": [0, -1], "held_object": null}, {"position": [1, 2], "orientation": [0, -1], "held_object": null}], "objects": [], "bonus_orders": [], "all_orders": [{"ingredients": ["onion"]}, {"ingredients": ["onion", "onion"]}, {"ingredients": ["onion", "onion", "onion"]}, {"ingredients": ["tomato"]}, {"ingredients": ["tomato", "tomato"]}, {"ingredients": ["tomato", "tomato", "tomato"]}, {"ingredients": ["onion", "tomato"]}, {"ingredients": ["onion", "onion", "tomato"]}, {"ingredients": ["onion", "tomato", "tomato"]}], "timestep": 0}'

In [50]:
# def check_clean_against_old(clean_player_data, old_team_data):
# states_for_player
# old_trials_team_states

def get_team_for_clean_worker(states_for_player):
    trial_num_results = {}
    for trial_num in layout_trials:

        old_trials_team_states = old_trials[old_trials['trial_id']==trial_num]['state'].to_numpy()

        is_team = True

        if len(states_for_player) != len(old_trials_team_states):
            is_team = False
        else:
            for i in range(len(states_for_player)):
                player_state = eval(states_for_player[i])
                team_state = json_eval(old_trials_team_states[i])


                ps_p1_pos = player_state['players'][0]['position']
                ps_p1_or = player_state['players'][0]['orientation']

                ps_p2_pos = player_state['players'][1]['position']
                ps_p2_or = player_state['players'][1]['orientation']

                ts_p1_pos = team_state['players'][0]['position']
                ts_p1_or = team_state['players'][0]['orientation']

                ts_p2_pos = team_state['players'][1]['position']
                ts_p2_or = team_state['players'][1]['orientation']

                if ps_p1_pos == ts_p1_pos and ps_p1_or == ts_p1_or:
                    if ps_p2_pos == ts_p2_pos and ps_p2_or == ts_p2_or:
        #                 is_team = True
                        good = 1
                    else:
                        is_team = False
                else:
                    is_team = False


        trial_num_results[trial_num] = is_team

    for trial_num in layout_trials:
        if trial_num_results[trial_num] == True:
            return trial_num
    return None


In [49]:
trial_num_results

{9: False,
 14: False,
 19: False,
 24: True,
 54: False,
 59: False,
 64: False,
 69: False,
 79: False,
 89: False,
 99: False,
 114: False}

In [54]:
fc_data = get_cleaned_fc_data_train()
unique_workerids = fc_data['workerid_num'].unique()
worker_id_to_team_num = {}
for w_id in unique_workerids:
    states_for_player = fc_data[fc_data['workerid_num']==w_id]['state'].to_numpy()
    team_num = get_team_for_clean_worker(states_for_player)
    worker_id_to_team_num[w_id] = team_num
    

     run  workerid_num  reward_norm_total  count
0  run-1             2              480.0   1204
1  run-1             4              280.0   1204
2  run-2            15              260.0   1199
3  run-2            17              300.0   1204
4  run-2            19              400.0   1204
5  run-2            22              300.0   1136


In [55]:
worker_id_to_team_num

{2: 14, 4: 24, 15: 79, 17: 89, 19: 99, 22: 114}

In [56]:
fc_data = get_cleaned_fc_data_test()
unique_workerids = fc_data['workerid_num'].unique()
worker_id_to_team_num = {}
for w_id in unique_workerids:
    states_for_player = fc_data[fc_data['workerid_num']==w_id]['state'].to_numpy()
    team_num = get_team_for_clean_worker(states_for_player)
    worker_id_to_team_num[w_id] = team_num
    

     run  workerid_num  reward_norm_total  count
0  run-1             1              380.0   1204
1  run-1             3              360.0   1204
2  run-2            10              440.0   1204
3  run-2            11              400.0   1204
4  run-2            12              160.0   1204
5  run-2            13              300.0   1204


In [57]:
worker_id_to_team_num

{1: 9, 3: 19, 10: 54, 11: 59, 12: 64, 13: 69}